<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project07B%20-%20Text%20Classification%20Deep%20Learning%20CNN%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN applied to sentiment analysis

Sentiment Analysis can be viewed as a classification task that determines whether a text expresses a positive or negative opinion. 

Instead of using tradictional classifiers (such as SVM), we will use a convolutional neural network (CNN) to train a classifier for the task of sentiment analysis. 

Deep CNN was initially developed for image classification. These networks, in particular, their convolutional layers,  are very efficient to recognize edges, patterns, context, and shapes. The convolutional layers can detect these features by using spatial dimensions smaller and deeper than the
original. 
Moreover, CNNs have been used successfully for many text classification tasks (such as sentiment analysis)


In particular, we will train a CNN to classify the reviews about movies. 
There are several availabel datasets of movie reviews (for example, https://www.kaggle.com/youssefamdouni/movie-review-classification-using-spacy/data). 

However, in this tutorial, we will use the dataset provided by nltk:




In [ ]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

print('Reviews:', len(movie_reviews.fileids()))
print('Labels:', movie_reviews.categories())

print('Number of words:', len(movie_reviews.words()) )
vocab=set(movie_reviews.words())
print('Size of the vocabulary (unique words):', len(vocab) )

# save the reviews into a list: X and their labels into another list: y
X=[] #list to save the texts
y=[] #list to save the labels

for file_id in movie_reviews.fileids(): #this traverse all movides
    category=movie_reviews.categories(file_id)
    label=category[0]
    tokens=list(movie_reviews.words(file_id))
    text=' '.join(str(word) for word in tokens)
    
    X.append(text)
    y.append(label)

X=list(X)
print(len(X),len(y))


Let's show some reviews:

In [ ]:
print(X[0])

## Training and test datasets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Encoding the classes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

## Baseline (tf-idf + svm)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import classification_report

from timeit import default_timer as timer

pipeline = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', svm.SVC()),  
])

start = timer()
pipeline.fit(X_train,y_train)
end = timer()
print("training time svm:", round(end-start, 3), "s")

print('pipeline (tf-idf + svm) was trained!!!')
start = timer()
y_pred = pipeline.predict(X_test)
end = timer()
print("prediction time svm:", round(end-start, 3), "s")

print("Baseline results using tf-idf (no cleaning) and svm")
print(classification_report(y_test, y_pred))


## Baseline 2 (tf-idf + MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

pipeline = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', MLPClassifier()),  
])

start = timer()
pipeline.fit(X_train,y_train)
end = timer()
print("training time MLP:", round(end-start, 3), "s")

print('pipeline (tf-idf + svm) was trained!!!')
start = timer()
y_pred = pipeline.predict(X_test)
end = timer()
print("prediction time MLP:", round(end-start, 3), "s")

print("Baseline results using tf-idf (no cleaning) and svm")
print(classification_report(y_test, y_pred))

## Training a CNN model

### Preprocessing

To prepare text data for our deep learning model, we transform each review into a sequence.
Every word in the review is mapped to an integer index and thus the sentence turns into a sequence of numbers.

To perform this transformation, keras provides the ```Tokenizer```

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train)

#we add a word index for the pad tokens
tokenizer.word_index['<PAD>'] = 0
print("Number of Documents in the training= {}".format(tokenizer.document_count))
print("Vocabulary size of the training= {}".format(len(tokenizer.word_index)))


**word_index** is a dictionary whose keys are the (unique) words in X_train and values are the indexes. 
Let's see some words and their indices:

In [ ]:
word='<PAD>'
print("Index for {}: {}".format(word, tokenizer.word_index[word]) )

word='<UNK>'
print("Index for {}: {}".format(word, tokenizer.word_index[word]) )
word='house'
print("Index for {}: {}".format(word, tokenizer.word_index[word]) )
word='cat'
print("Index for {}: {}".format(word, tokenizer.word_index[word]) )


### Exercise:
Why does the following cell throw an error?

In [ ]:
# word='Cat'
# print("Index for {}: {}".format(word, tokenizer.word_index[word]) )


The following cell shows the word with the largests index. It also show the index for the padding token (0) and the index for unknown words (=1). 


In [ ]:
print("Word with the largest index", max([(k, v) for k, v in tokenizer.word_index.items()], key = lambda x:x[1]))
print("Word with the lowest index", min([(k, v) for k, v in tokenizer.word_index.items()], key = lambda x:x[1]))
print("Word for unkown words",('<UNK>', tokenizer.word_index['<UNK>']) )


### Transform texts to sequence of integers

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

#print(X_train[0].split())
print("Review: ", X_train[0])

print("Sequence of integers: ", train_sequences[0])
print("Length of the sequence:", len(train_sequences[0]), ' tokens')


## Exercise:
Show the sequence of integers for the review with index 5 from the training.
What is the lenght of this review?



In [ ]:
# write here your code

Please, show more sequences and their corresponding lengths:

In [ ]:
# write here your code


Each word in a review will be represented as a vector (which is randomly created or from a word embedding model) of a given dimension (for example, 300).
Therefore, each review will be represented as a matrix with:
- number of rows: the number of tokens in the review
- number of columns: 300

We need that all sequence have the same number of tokens. How can we choose this number?


In [ ]:
import pandas as pd

train_lens = [len(s) for s in train_sequences]
df = pd.DataFrame(train_lens)
df.head()

The mean of lengths is:

In [ ]:
df.mean()

The **describe** method (of a dataframe) lets us show more statistics:

In [ ]:
df.describe()

The following graph shows the size distribution (number of tokens) of the reviews. We can see that most reviews have less than 1000 tokens. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
test_lens = [len(s) for s in test_sequences]

fig, ax = plt.subplots(1,2, figsize=(12, 6))
h1 = ax[0].hist(train_lens)
h2 = ax[1].hist(test_lens)
ax[0].title.set_text('Sequence length distribution in the training dataset')
ax[1].title.set_text('Sequence length distribution in the test dataset')
ax[0].set_xlabel('Sequence Length')
ax[0].set_ylabel('Number of reviews')
ax[1].set_xlabel('Sequence Length')
ax[1].set_ylabel('Number of reviews')



### Padding 
Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length.
For reviews which are smaller than this length, we pad them with zeros which longer ones are truncated

In [ ]:
MAX_SEQUENCE_LENGTH = 1000 #1500

In [ ]:
# pad dataset to a maximum review length in words
from tensorflow.keras.preprocessing import sequence

train_seq_pad = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_seq_pad = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_seq_pad.shape, test_seq_pad.shape

Let's show some examples after padding:

For example, the sequence with index 1 has a lenght of 697 tokens. 
The **pad_sequences** has added 1000-697 padding tokens at the beginning of the sequence. 


In [ ]:
print("length of the first sentence", len(train_sequences[0]), "after padding:", len(train_seq_pad[0]))
#print(X_train[0])
print(len(train_sequences[1]))

print(train_sequences[1])
print()
print(len(train_seq_pad[1]))
print(train_seq_pad[1])

The method adds the padding tokens at the beginning, by default. You can change this behaviour using the argument: **padding = post**

In [ ]:
train_seq_pad = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')


In [ ]:
print(len(train_seq_pad[1]))
print(train_sequences[1])
print(train_seq_pad[1])

Some sequences are longer than the maximum length (1000). 
By default, the **pad_sequence** will remove (trunk) the tokens at the beginning of the sequence. 


In [ ]:
print(len(train_seq_pad[0]))
print(train_sequences[0])
print(train_seq_pad[0])

You can remove the tokens at the end by using the parameter **trunking = post**

In [ ]:
train_seq_pad = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating='post')
print(len(train_seq_pad[0]))
print(train_sequences[0])
print(train_seq_pad[0])

You already know how to use padding and trunking. We will apply the default behaviour (pre padding and pre truncating)

In [ ]:
train_seq_pad = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_seq_pad = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Here you can find more information about the **pad_sequences** method


https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

## Prepare the Model

Since textual data is a sequence of words, we utilize ```1D``` convolutions to scan through the sentences.
The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [ ]:
import tensorflow as tf
tf.random.set_seed(1)

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding




In [ ]:
VOCAB_SIZE = len(tokenizer.word_index)

In [ ]:
EMBED_SIZE = 300
EPOCHS=20
BATCH_SIZE=16

In [ ]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Model Training

In [ ]:
from keras.callbacks import EarlyStopping
#earlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


# Fit the model
history=model.fit(train_seq_pad, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1, #callbacks=[earlyStopping]
          )

In [ ]:
model.metrics_names

## Model Evaluation

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_seq_pad, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
predictions=model.predict(test_seq_pad) 
# print(predictions)

predictions.reshape(-1)
predictions = [1 if item >= 0.5 else 0 for item in predictions]
predictions[:10]

In [ ]:
print(classification_report(y_test, predictions))


## Exercises

- 1) What does it happen if you add more convolutional layers to your model? Do the results improve?
- 2) What does it happen if the network is inistialized with pre-trained word embeddings instead of using random initialization? Do the results improve?
- 3) Plot the learning curves. 